# 1. Getting Started

Author: Arianna Basile, MRC Toxicology Unit, University of Cambridge

__________________________________________________________________________________________________________________
In this tutorial:
<ul>
    <li>You will use the library cobrapy to import a metabolic reconstruction (1.1) </li>
    <li>Inspect the reactions of your model (1.2)</li>
        <li>Inspect the metabolites in your model (1.3) </li>
        <li>Inspect the genes in your model (1.4) </li>
        <li>Perform knockout in silico experiment (1.4.1) </li>
        <li>Challenge yourself with some exercises</li>
        </ul>

In [80]:
#import the needed packages
import cobra

#enable autocompleting with tab
%config Completer.use_jedi = False


## 1.1 Import a reconstruction

The Systems Biology Markup Language is an XML-based standard format for distributing models which has support for COBRA models through the FBC extension version 2.

Cobrapy has native support for reading and writing SBML with FBCv2. Please note that all id’s in the model must conform to the SBML SID requirements in order to generate a valid SBML file.

Let's download and import the model of <a href="http://bigg.ucsd.edu/search?query=Saccharomyces+cerevisiae+S288C"> Saccharomyces cerevisiae </a>

In [49]:
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

The reactions, metabolites, and genes attributes of the cobrapy model are a special type of list called a cobra.DictList, and each one is made up of cobra.Reaction, cobra.Metabolite and cobra.Gene objects respectively.

In [7]:
print(len(model_yeast.reactions))
print(len(model_yeast.metabolites))
print(len(model_yeast.genes))

1577
1226
905


When using Jupyter notebook this type of information is rendered as a table.

In [8]:
model_yeast

Name,iMM904
Memory address,0x07fac9bb33760
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


Just like a regular list, objects in the DictList can be retrieved by index. For example, to get the 3rd reaction in the model (at index 2nd because of 0-indexing):

In [9]:
model_yeast.reactions[2]

Reaction identifier,13BGHe
Name,Exo 1 3 beta glucan glucohydrase
Memory address,0x07fac9bd190a0
Stoichiometry,13BDglcn_e + h2o_e --> glc__D_e 1 3 beta D Glucan C6H10O5 + H2O H2O --> D-Glucose
GPR,YOR190W or YDR261C or YLR300W
Lower bound,0.0
Upper bound,999999.0


Additionally, items can be retrieved by their id using the DictList.get_by_id() function. For example, to get the cytosolic atp metabolite object (the id is “atp_c”), we can do the following:

## 1.2 Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI.

In [50]:
pgi = model_yeast.reactions.get_by_id("PGI")
pgi

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x07facc93743d0
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,YBR196C
Lower bound,-999999.0
Upper bound,999999.0


We can view the full name and reaction catalyzed as strings.

In [12]:
print(pgi.name)
print(pgi.reaction)

Glucose-6-phosphate isomerase
g6p_c <=> f6p_c


We can also view reaction upper and lower bounds. Because the pgi.lower_bound < 0, and pgi.upper_bound > 0, pgi is reversible.

In [14]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-999999.0 < pgi < 999999.0
True


The lower and upper bound of reactions can also be modified, and the reversibility attribute will automatically be updated. The preferred method for manipulating bounds is using reaction.bounds, e.g.

In [15]:
old_bounds = pgi.bounds
pgi.bounds = (0, 1000.0)
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print("Reversibility after modification:", pgi.reversibility)
pgi.bounds = old_bounds
print("Reversibility after resetting:", pgi.reversibility)

0 < pgi < 1000.0
Reversibility after modification: False
Reversibility after resetting: True


We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [16]:
pgi.check_mass_balance()

{}

In order to add a metabolite, we pass in a dict with the metabolite object and its coefficient

In [18]:
pgi.add_metabolites({model_yeast.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

The reaction is no longer mass balanced

In [19]:
pgi.check_mass_balance()

{'charge': -1.0, 'H': -1.0}

We can remove the metabolite, and the reaction will be balanced once again.



In [21]:
pgi.subtract_metabolites({model_yeast.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


## 1.3 Metabolites

We will consider cytosolic atp as our metabolite, which has the id "atp_c" in our test model.



In [22]:
atp = model_yeast.metabolites.get_by_id("atp_c")
atp

Metabolite identifier,atp_c
Name,ATP C10H12N5O13P3
Memory address,0x07facb996c670
Formula,C10H12N5O13P3
Compartment,c
In 149 reaction(s),"URIK1, TRPTRS, FACOAL161, GSNK, ADK1, PI3P5K_SC, CBPS, FACOAL181, FACOAL140, FACOAL180, NMNAT, ACCOAC, HEX4, INSK, BIOMASS_SC5_notrace, PMPK, TYRTRS, ACS, PGK, PI3P4K_SC, SLCBK2, DHAK, NNATr,..."


We can print out the metabolite name and compartment (cytosol in this case) directly as string.



In [23]:
print(atp.name)
print(atp.compartment)

ATP C10H12N5O13P3
c


We can see that ATP is a charged molecule in our model.



In [24]:
atp.charge

-4

We can see the chemical formula for the metabolite as well.



In [26]:
print(atp.formula)

C10H12N5O13P3


## 1.4 Genes

The gene_reaction_rule is a boolean representation of the gene requirements for this reaction to be active as described in <a href="https://www.nature.com/articles/nprot.2011.308">Schellenberger et al 2011 Nature Protocols 6(9):1290-307</a>.

The GPR is stored as the GPR class in the gpr for a Reaction. A string representation of it is stored as the gene_reaction_rule for a Reaction object.

In [31]:
gpr_string = pgi.gene_reaction_rule
print(gpr_string)

YBR196C


Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [33]:
pgi_gene = model_yeast.genes.get_by_id("YBR196C")
pgi_gene

Gene identifier,YBR196C
Name,PGI1
Memory address,0x07fac9bc6a460
Functional,True
In 3 reaction(s),"PGI, G6PI, G6PI3"


Let's check this gene also on KEGG database to assess the consistency of the metabolic reconstruction --> <a href="https://www.genome.jp/entry/sce:YBR196C"> HERE </a> 

### 1.4.1 Simulating Knockouts

The delete_model_genes function will evaluate the GPR and set the upper and lower bounds to 0 if the reaction is knocked out.

In [51]:
print("before KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))


cobra.manipulation.delete_model_genes(
    model_yeast, ["YBR196C"])
print("after KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))

before KO: -999999 < flux_PGI < 999999
after KO:    0 < flux_PGI <    0


Quite often, one wants to make small changes to a model and evaluate the impacts of these. For example, we may want to knock-out all reactions sequentially, and see what the impact of this is on the objective function. One way of doing this would be to create a new copy of the model before each knock-out with model.copy(). However, even with small models, this is a very slow approach as models are quite complex objects. Better then would be to do the knock-out, optimizing and then manually resetting the reaction bounds before proceeding with the next reaction. Since this is such a common scenario however, cobrapy allows us to use the model as a context, to have changes reverted automatically.

In [63]:
#we import the model again to delete the edits on genes knockouts
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

#we impose knockouts of reactions
model_yeast.optimize()
print(model_yeast.objective.value)
for reaction in model_yeast.reactions:
    with model_yeast as model_yeast:  #not to edit the original model
        reaction.knock_out()
        model_yeast.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model_yeast.objective.value))

0.28786570370401665
CITtcp blocked (bounds: (0, 0)), new growth rate 0.287866
13BGH blocked (bounds: (0, 0)), new growth rate 0.287866
13BGHe blocked (bounds: (0, 0)), new growth rate 0.287866
13GS blocked (bounds: (0, 0)), new growth rate 0.000000
16GS blocked (bounds: (0, 0)), new growth rate 0.287866
23CAPPD blocked (bounds: (0, 0)), new growth rate 0.287866
2DDA7Ptm blocked (bounds: (0, 0)), new growth rate 0.287866
2DHPtm blocked (bounds: (0, 0)), new growth rate 0.287866
2DOXG6PP blocked (bounds: (0, 0)), new growth rate 0.287866
2HBO blocked (bounds: (0, 0)), new growth rate 0.287866
2HBt2 blocked (bounds: (0, 0)), new growth rate 0.287866
2HMHMBQMTm blocked (bounds: (0, 0)), new growth rate 0.287866
2HP6MPMOm blocked (bounds: (0, 0)), new growth rate 0.287866
2HPMBQMTm blocked (bounds: (0, 0)), new growth rate 0.287866
2HPMMBQMOm blocked (bounds: (0, 0)), new growth rate 0.287866
2MBACt blocked (bounds: (0, 0)), new growth rate 0.287866
EX_epistest_SC_e blocked (bounds: (0, 0))

ALPHNH blocked (bounds: (0, 0)), new growth rate 0.287866
AMAOTr blocked (bounds: (0, 0)), new growth rate 0.287866
ADSL1r blocked (bounds: (0, 0)), new growth rate 0.000000
AMETt2 blocked (bounds: (0, 0)), new growth rate 0.287866
3DH5HPBMTm blocked (bounds: (0, 0)), new growth rate 0.287866
AMETtm blocked (bounds: (0, 0)), new growth rate 0.287866
AMID blocked (bounds: (0, 0)), new growth rate 0.287866
AMID2 blocked (bounds: (0, 0)), new growth rate 0.287866
AMID3 blocked (bounds: (0, 0)), new growth rate 0.287866
ADSL2r blocked (bounds: (0, 0)), new growth rate 0.000000
ADSS blocked (bounds: (0, 0)), new growth rate 0.000000
AMPN blocked (bounds: (0, 0)), new growth rate 0.287866
AMPDA blocked (bounds: (0, 0)), new growth rate 0.287866
3DH5HPBtm blocked (bounds: (0, 0)), new growth rate 0.287866
AMPtn blocked (bounds: (0, 0)), new growth rate 0.287866
ANPRT blocked (bounds: (0, 0)), new growth rate 0.000000
ATPS3v blocked (bounds: (0, 0)), new growth rate 0.287866
3DSPHR blocked (bo

CO2t blocked (bounds: (0, 0)), new growth rate 0.218775
CO2tg blocked (bounds: (0, 0)), new growth rate 0.287866
CO2tm blocked (bounds: (0, 0)), new growth rate 0.000000
CO2tn blocked (bounds: (0, 0)), new growth rate 0.287866
CO2tp blocked (bounds: (0, 0)), new growth rate 0.287866
CHTNDA blocked (bounds: (0, 0)), new growth rate 0.287866
CHTNS blocked (bounds: (0, 0)), new growth rate 0.287866
CITt2r blocked (bounds: (0, 0)), new growth rate 0.287866
CO2tv blocked (bounds: (0, 0)), new growth rate 0.287866
COAHLp blocked (bounds: (0, 0)), new growth rate 0.287866
3OPHB_5tm blocked (bounds: (0, 0)), new growth rate 0.287866
COAtim blocked (bounds: (0, 0)), new growth rate 0.287654
CITtam blocked (bounds: (0, 0)), new growth rate 0.284898
CITtap blocked (bounds: (0, 0)), new growth rate 0.287866
COAtn blocked (bounds: (0, 0)), new growth rate 0.287866
4ABUTNtm blocked (bounds: (0, 0)), new growth rate 0.287866
COUCOAFm blocked (bounds: (0, 0)), new growth rate 0.287866
CPPPGO blocked (

D_LACt2m blocked (bounds: (0, 0)), new growth rate 0.287866
D_LACtm blocked (bounds: (0, 0)), new growth rate 0.287866
E4HGLUtm blocked (bounds: (0, 0)), new growth rate 0.287866
ERGSTt blocked (bounds: (0, 0)), new growth rate 0.287866
ERGSTter blocked (bounds: (0, 0)), new growth rate 0.000000
ERGTETROLter blocked (bounds: (0, 0)), new growth rate 0.000000
E4HGLUtp blocked (bounds: (0, 0)), new growth rate 0.287866
E4Ptm blocked (bounds: (0, 0)), new growth rate 0.287866
ETHAK blocked (bounds: (0, 0)), new growth rate 0.287866
5AOPt2 blocked (bounds: (0, 0)), new growth rate 0.287866
ETHAPT_SC blocked (bounds: (0, 0)), new growth rate 0.287866
ETHAt blocked (bounds: (0, 0)), new growth rate 0.287866
ETOHt blocked (bounds: (0, 0)), new growth rate 0.231101
ETOHtm blocked (bounds: (0, 0)), new growth rate 0.287866
ECOAH11p blocked (bounds: (0, 0)), new growth rate 0.287866
EX_ergst_e blocked (bounds: (0, 0)), new growth rate 0.287866
EX_ergstest_SC_e blocked (bounds: (0, 0)), new growt

/Users/arianna/miniconda3/envs/carveme2/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


...

KARA2im blocked (bounds: (0, 0)), new growth rate 0.000000
KYN blocked (bounds: (0, 0)), new growth rate 0.287866
KYN3OX blocked (bounds: (0, 0)), new growth rate 0.287866
Kt2r blocked (bounds: (0, 0)), new growth rate 0.287866
LALDO3 blocked (bounds: (0, 0)), new growth rate 0.287866
LANOSTAT_SC blocked (bounds: (0, 0)), new growth rate 0.287866
ICDHyr blocked (bounds: (0, 0)), new growth rate 0.000000
ICDHym blocked (bounds: (0, 0)), new growth rate 0.287060
ICDHyp blocked (bounds: (0, 0)), new growth rate 0.287866
ICL blocked (bounds: (0, 0)), new growth rate 0.287510
LANOSTESTH_SC blocked (bounds: (0, 0)), new growth rate 0.287866
FOLD3m blocked (bounds: (0, 0)), new growth rate 0.287866
LANOSTESTH_SCe blocked (bounds: (0, 0)), new growth rate 0.287866
LANOSTt blocked (bounds: (0, 0)), new growth rate 0.287866
LCADi blocked (bounds: (0, 0)), new growth rate 0.287866
LCYSTintv blocked (bounds: (0, 0)), new growth rate 0.287866
LEUTA blocked (bounds: (0, 0)), new growth rate 0.000000

MIP2C326PLC_SC blocked (bounds: (0, 0)), new growth rate 0.287866
NAPRTm blocked (bounds: (0, 0)), new growth rate 0.287866
NAt3_1 blocked (bounds: (0, 0)), new growth rate 0.287866
NDP1 blocked (bounds: (0, 0)), new growth rate 0.287866
NDP3 blocked (bounds: (0, 0)), new growth rate 0.287866
NDP3g blocked (bounds: (0, 0)), new growth rate 0.287866
NDP4 blocked (bounds: (0, 0)), new growth rate 0.287866
MIP2CS124_SC blocked (bounds: (0, 0)), new growth rate 0.287866
MIP2CS126_SC blocked (bounds: (0, 0)), new growth rate 0.287866
NDP7 blocked (bounds: (0, 0)), new growth rate 0.287866
MIP2CS224_SC blocked (bounds: (0, 0)), new growth rate 0.287866
FUMm blocked (bounds: (0, 0)), new growth rate 0.287247
MIP2CS226_SC blocked (bounds: (0, 0)), new growth rate 0.287866
MIP2CS324_SC blocked (bounds: (0, 0)), new growth rate 0.287866
MIP2CS326_SC blocked (bounds: (0, 0)), new growth rate 0.287866
MIPC124PLC_SC blocked (bounds: (0, 0)), new growth rate 0.287866
NDPK1 blocked (bounds: (0, 0)), 

PIt2n blocked (bounds: (0, 0)), new growth rate 0.287866
PIt2p blocked (bounds: (0, 0)), new growth rate 0.287866
PIt2r blocked (bounds: (0, 0)), new growth rate -0.000000
PAPtm blocked (bounds: (0, 0)), new growth rate 0.287866
PIt2v blocked (bounds: (0, 0)), new growth rate 0.287866
PAtm_SC blocked (bounds: (0, 0)), new growth rate 0.287866
PPPG9tm blocked (bounds: (0, 0)), new growth rate 0.287866
PC blocked (bounds: (0, 0)), new growth rate 0.264200
PCDAGAT blocked (bounds: (0, 0)), new growth rate 0.287866
PIt5m blocked (bounds: (0, 0)), new growth rate 0.287866
PLBP1I_SCe blocked (bounds: (0, 0)), new growth rate 0.287866
PLBPC_SC blocked (bounds: (0, 0)), new growth rate 0.287866
PDE1 blocked (bounds: (0, 0)), new growth rate 0.287866
PLBPC_SCe blocked (bounds: (0, 0)), new growth rate 0.287866
PPPGOm blocked (bounds: (0, 0)), new growth rate 0.287866
PLD_SC blocked (bounds: (0, 0)), new growth rate 0.287866
PMANM blocked (bounds: (0, 0)), new growth rate 0.000000
PMDPHT blocked

SHSL1 blocked (bounds: (0, 0)), new growth rate 0.287866
TCHOLAabcv blocked (bounds: (0, 0)), new growth rate 0.287866
THFATm blocked (bounds: (0, 0)), new growth rate 0.287866
THFGLUS blocked (bounds: (0, 0)), new growth rate 0.287866
THIORDXi blocked (bounds: (0, 0)), new growth rate 0.287866
THIORDXm blocked (bounds: (0, 0)), new growth rate 0.287866
THIORDXni blocked (bounds: (0, 0)), new growth rate 0.287866
THIORDXp blocked (bounds: (0, 0)), new growth rate 0.287866
SHSL4r blocked (bounds: (0, 0)), new growth rate 0.287866
SLCBK1 blocked (bounds: (0, 0)), new growth rate 0.287866
PRPPtm blocked (bounds: (0, 0)), new growth rate 0.287866
THMDPe blocked (bounds: (0, 0)), new growth rate 0.287866
THMDt2 blocked (bounds: (0, 0)), new growth rate 0.287866
THMP blocked (bounds: (0, 0)), new growth rate 0.287866
PSCVT blocked (bounds: (0, 0)), new growth rate 0.000000
THMPPt2m blocked (bounds: (0, 0)), new growth rate 0.287866
THMPe blocked (bounds: (0, 0)), new growth rate 0.287866
THR

PUNP5 blocked (bounds: (0, 0)), new growth rate 0.287866
PUNP6 blocked (bounds: (0, 0)), new growth rate 0.287866
PUNP7 blocked (bounds: (0, 0)), new growth rate 0.287866
PYAM5PO blocked (bounds: (0, 0)), new growth rate 0.287866
PYDAMK blocked (bounds: (0, 0)), new growth rate 0.287866
PYDXK blocked (bounds: (0, 0)), new growth rate 0.287866
PYDXNK blocked (bounds: (0, 0)), new growth rate 0.287866
PYDXNO blocked (bounds: (0, 0)), new growth rate 0.287866
PYDXO blocked (bounds: (0, 0)), new growth rate 0.287866
PYK blocked (bounds: (0, 0)), new growth rate 0.052448
PYNP2r blocked (bounds: (0, 0)), new growth rate 0.287866
PYR5CDm blocked (bounds: (0, 0)), new growth rate 0.287866
PYRDC blocked (bounds: (0, 0)), new growth rate 0.191654


In [65]:
#we import the model again to delete the edits on genes knockouts
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

#we impose knockouts on genes directly
model_yeast.optimize()
print(model_yeast.objective.value)
for gene in model_yeast.genes:
    with model_yeast as model_yeast:  #not to edit the original model
        gene.knock_out()
        model_yeast.optimize()
        print('%s, new growth rate %f' %
              (gene.id, model_yeast.objective.value))

0.28786570370401665
YHR104W, new growth rate 0.287866
YDR368W, new growth rate 0.287866
YGR282C, new growth rate 0.287866
YOL086C, new growth rate 0.287866
YLR300W, new growth rate 0.287866
YFR055W, new growth rate 0.287866
YGL184C, new growth rate 0.287866
YDR261C, new growth rate 0.287866
YDL168W, new growth rate 0.287866
YOR190W, new growth rate 0.287866
YOL030W, new growth rate 0.287866
YLR343W, new growth rate 0.287866
YNL247W, new growth rate 0.287866
YMR303C, new growth rate 0.287866
YGR155W, new growth rate 0.287843
YGR032W, new growth rate 0.287866
YOL132W, new growth rate 0.287866
YGL256W, new growth rate 0.287866
YBR145W, new growth rate 0.287866
YCR034W, new growth rate 0.000000
YMR215W, new growth rate 0.287866
YMR307W, new growth rate 0.287866
YLR342W, new growth rate 0.287866
YMR306W, new growth rate 0.287866
YBR068C, new growth rate 0.287866
YMR083W, new growth rate 0.287866
YMR318C, new growth rate 0.287866
YBR036C, new growth rate 0.287866
YDR046C, new growth rate 0.2

YER060W, new growth rate 0.287866
YGL186C, new growth rate 0.287866
YDL141W, new growth rate 0.287866
YKL140W, new growth rate 0.287866
YOL064C, new growth rate 0.285150
YLL012W, new growth rate 0.287866
YIL124W, new growth rate 0.287866
YAL060W, new growth rate 0.287866
YDR226W, new growth rate 0.000000
YLR020C, new growth rate 0.287866
YGR065C, new growth rate 0.287866
YIL013C, new growth rate 0.287866
YER170W, new growth rate 0.287866
YGR286C, new growth rate 0.287866
YNL280C, new growth rate 0.000000
YOL052C, new growth rate 0.287866
YOR011W, new growth rate 0.287866
YHR042W, new growth rate 0.000000
YBR184W, new growth rate 0.287866
YJL005W, new growth rate 0.287866
YMR015C, new growth rate 0.287866
YNL111C, new growth rate 0.287858
YJR105W, new growth rate 0.287802
YNR019W, new growth rate 0.287866
YKL150W, new growth rate 0.287866
YIL043C, new growth rate 0.287866
YLR189C, new growth rate 0.287866
YDR400W, new growth rate 0.287866
YGL012W, new growth rate 0.000000
YAL022C, new g

YER178W, new growth rate 0.281522
YNR041C, new growth rate 0.287866
YBR035C, new growth rate 0.287866
YDL182W, new growth rate 0.287866
YNL318C, new growth rate 0.287866
YGR147C, new growth rate 0.287866
YDL040C, new growth rate 0.287866
YGR192C, new growth rate 0.287866
YDL131W, new growth rate 0.287866
YKR093W, new growth rate 0.287866
YPL273W, new growth rate 0.287866
YJL052W, new growth rate 0.287866
YGR007W, new growth rate 0.287866
YJR009C, new growth rate 0.287866
YLL062C, new growth rate 0.287866
YGR157W, new growth rate 0.000000
YMR205C, new growth rate 0.071791
YER141W, new growth rate 0.287866
YDR408C, new growth rate 0.000000
YGR240C, new growth rate 0.071791
YKR067W, new growth rate 0.287866
YBL011W, new growth rate 0.287866
YDR376W, new growth rate 0.287866
YIL107C, new growth rate 0.287866
YPL252C, new growth rate 0.287866
YOL136C, new growth rate 0.287866
YEL011W, new growth rate 0.287866
YER081W, new growth rate 0.287866
YMR189W, new growth rate 0.287866
YPL172C, new g

YFL001W, new growth rate 0.287866
YBR256C, new growth rate 0.000000
YGR287C, new growth rate 0.287866
YIL066C, new growth rate 0.287866
YGR292W, new growth rate 0.287866
YNL292W, new growth rate 0.287866
YER060W_A, new growth rate 0.287866
YER070W, new growth rate 0.287866
YJL026W, new growth rate 0.287866
YEL017C_A, new growth rate 0.287866
YIL172C, new growth rate 0.287866
YGR180C, new growth rate 0.287866
YGR289C, new growth rate 0.287866
YNL129W, new growth rate 0.287866
YCR024C_A, new growth rate 0.287866
YJL121C, new growth rate 0.286536
YHR039C_A, new growth rate 0.287866
YDL247W, new growth rate 0.287866
YOR095C, new growth rate 0.000000
YJR160C, new growth rate 0.287866
YBR298C, new growth rate 0.287866
YDR322C_A, new growth rate 0.287866
YNR050C, new growth rate 0.000000
YML081C_A, new growth rate 0.236970
YIR034C, new growth rate 0.000000
YLR348C, new growth rate 0.287866
YJR010W, new growth rate 0.287866
YDL055C, new growth rate 0.000000
YER003C, new growth rate 0.000000
YO

### Excercises

1) Why knocking out genes or reactions give different output information? 

2) Can you verify the consistency between gene and reactions knockouts results using a gene or a reaction of your choice?

3) Can you verify your hypothesis from the previous excercise using relevant databases (e.g. KEGG and the SGD)?

4) Do you expect these results to change if we change the medium where we are growing our yeast model? 

### solution exercise 2

In [75]:
#from the results of the second chunk of simulations, I know that YJL167W is an essential gene
#I check with genes.get_by_id and I identify the reactions catalysed by this enzyme
model_yeast.genes.get_by_id("YJL167W")


Gene identifier,YJL167W
Name,ERG20
Memory address,0x07facbcfeb9a0
Functional,True
In 2 reaction(s),"GRTT, DMATT"


In [86]:
#when I selectively knockout the reaction GRTT, I get no growth of my model
with model_yeast as model_yeast:  #not to edit the original model
    grtt = model_yeast.reactions.get_by_id("GRTT")
    grtt.knock_out()
    model_yeast.optimize()
    print('%s blocked (bounds: %s), new growth rate %f' %
            (grtt.id, str(reaction.bounds), model_yeast.objective.value))
    
#when I selectively knockout the reaction DMATT, I get no growth of my model
with model_yeast as model_yeast:  #not to edit the original model
    dmatt = model_yeast.reactions.get_by_id("DMATT")
    dmatt.knock_out()
    model_yeast.optimize()
    print('%s blocked (bounds: %s), new growth rate %f' %
            (dmatt.id, str(reaction.bounds), model_yeast.objective.value))



GRTT blocked (bounds: (0.0, 999999.0)), new growth rate 0.000000
DMATT blocked (bounds: (0.0, 999999.0)), new growth rate 0.000000


### solution exercise 3
When we look for YJL167W in the Saccharomyces genome database --> <a href="https://www.yeastgenome.org/locus/S000003703"> HERE </a>, we find out that YJL167W is an essential gene

### solution exercise 4
Yes, we will go deeper in this in the next notebook :)